In [1]:
import pandas as pd

Model Traning

In [2]:
df= pd.read_csv('./data/diamonds.csv')
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
df= df.drop(labels=['Unnamed: 0'],axis=1)

In [ ]:
Y

In [4]:
##Independent and dependent features
X=df.drop(labels=['price'],axis=1)
Y=df['price']

In [5]:
# Define which column should be ordinal-encoded and which should be scaled
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [6]:
#define the costom ranking for each ordinal variable
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [7]:
from sklearn.impute import SimpleImputer
##MEAN MEDIAN MOST FREQUENT
##for filling null values
##Handling missing values

In [8]:
from sklearn.preprocessing import StandardScaler
##Need to reach global minimum
## Handling Feature Scaling

In [9]:
from sklearn.preprocessing import OrdinalEncoder
#Ordinal encoding , mapping

In [10]:
from sklearn.pipeline import Pipeline
#pipelines

In [11]:
from sklearn.compose import ColumnTransformer

In [12]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
        
    ]

)

## Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('impuer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler()) ## After ordinalencoder we will get numerical values and them we have to scale it
    ]
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [13]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.30,random_state=33)

In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.966886,0.174186,-0.653685,-1.134489,-1.088617,-1.082475,0.984132,0.238108,1.187741
1,1.121469,0.662259,-0.653685,1.121742,1.130137,1.210629,0.984132,0.824304,-0.636730
2,0.235500,0.801709,-1.100909,0.354802,0.376108,0.465019,-0.805320,-0.934284,-0.028573
3,0.003461,1.359507,-0.206460,0.122936,0.142098,0.296201,-0.805320,-1.520479,-1.244887
4,0.214406,0.592535,0.240764,0.328048,0.350107,0.408746,-1.700046,-0.348088,-0.636730


In [16]:
## Model Traing
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
regression=LinearRegression()
regression.fit(X_train,Y_train)

LinearRegression()

In [18]:
regression.coef_

array([5097.69873634, -113.45717147,  -58.32362962, -979.50678811,
         26.44991755,   -6.45838613,  135.12920309, -555.19958617,
        820.82865003])

In [19]:
regression.intercept_

np.float64(3929.972588590498)

In [21]:
import numpy as np
def evalute_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [22]:
## Train multiple Model

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]


for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)
    
    ##Make prediction
    Y_pred=model.predict(X_test)
    mae,rmse,r2_square=evalute_model(Y_test,Y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n') 

LinearRegression
Model Training Performance
RMSE: 1233.4322134040483
MAE: 812.8829031332132
R2 score 90.48319863562268


Lasso
Model Training Performance
RMSE: 1233.5369984617244
MAE: 813.9721090739673
R2 score 90.48158158536586


Ridge
Model Training Performance
RMSE: 1233.4310142423446
MAE: 812.9783718151258
R2 score 90.48321714037401


ElasticNet
Model Training Performance
RMSE: 1607.0687182615645
MAE: 1066.6178760008197
R2 score 83.84416806590013




In [23]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']